<a href="https://colab.research.google.com/github/angegiulio/IFC-Draw-Door/blob/main/ifcpython_Draw_door.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install ifcopenshell
!pip install -U kaleido



In [25]:
import ifcopenshell

# Make sure your file is named correctly
ifc_file = ifcopenshell.open("022_GHA_3D_OG12_000_YYY_ARLE_41.ifc")

# Get the first IfcDoor
door = ifc_file.by_type("IfcDoor")[0]


In [26]:
import ifcopenshell.geom

settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)

shape = ifcopenshell.geom.create_shape(settings, door)
geometry = shape.geometry


In [30]:
# Extract raw vertex and face data from the shape
verts = geometry.verts
faces = geometry.faces

# ✅ Convert flat vertex list into list of (x, y, z) tuples
vertices = [(verts[i], verts[i+1], verts[i+2]) for i in range(0, len(verts), 3)]

# Split x, y, z lists for Mesh3d
x = [v[0] for v in vertices]
y = [v[1] for v in vertices]
z = [v[2] for v in vertices]

# Split faces for Mesh3d
i = faces[0::3]
j = faces[1::3]
k = faces[2::3]


In [31]:
import plotly.graph_objects as go

# --- 1. Solid mesh (light-colored base) ---
solid = go.Mesh3d(
    x=x, y=y, z=z,
    i=i, j=j, k=k,
    color='lightgrey',
    opacity=1.0,
    flatshading=True,
    lighting=dict(ambient=0.4, diffuse=0.8, specular=0.1),
    lightposition=dict(x=100, y=200, z=0),
    showscale=False
)

# --- 2. Wireframe overlay from triangle edges ---
lines = []
for idx in range(0, len(faces), 3):
    tri = [faces[idx], faces[idx+1], faces[idx+2]]
    edges = [(tri[0], tri[1]), (tri[1], tri[2]), (tri[2], tri[0])]
    for start, end in edges:
        x0, y0, z0 = vertices[start]
        x1, y1, z1 = vertices[end]
        lines.append(((x0, x1), (y0, y1), (z0, z1)))

wireframe_lines = [
    go.Scatter3d(
        x=edge[0], y=edge[1], z=edge[2],
        mode='lines',
        line=dict(color='black', width=1),
        showlegend=False
    )
    for edge in lines
]

# --- 3. Create figure with solid + lines ---
fig = go.Figure(data=[solid] + wireframe_lines)

# --- 4. Set up front-view camera ---
camera = dict(
    eye=dict(x=0, y=-3, z=1.5),
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0)
)

# --- 5. Layout for clarity ---
fig.update_layout(
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        aspectmode='data',
        bgcolor='white'  # clean background
    ),
    margin=dict(l=0, r=0, t=0, b=0),
    scene_camera=camera,
    paper_bgcolor='white'
)

fig.show()


In [32]:
# Save image
fig.write_image("ifc_door_front_view.png", width=800, height=600)
